In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hand-gestures-recognition/dataset/test_set/two/two_gesture260.png
/kaggle/input/hand-gestures-recognition/dataset/test_set/two/two_gesture438.png
/kaggle/input/hand-gestures-recognition/dataset/test_set/two/two_gesture428.png
/kaggle/input/hand-gestures-recognition/dataset/test_set/two/two_gesture697.png
/kaggle/input/hand-gestures-recognition/dataset/test_set/two/two_gesture550.png
/kaggle/input/hand-gestures-recognition/dataset/test_set/two/two_gesture603.png
/kaggle/input/hand-gestures-recognition/dataset/test_set/two/two_gesture417.png
/kaggle/input/hand-gestures-recognition/dataset/test_set/two/two_gesture220.png
/kaggle/input/hand-gestures-recognition/dataset/test_set/two/two_gesture343.png
/kaggle/input/hand-gestures-recognition/dataset/test_set/two/two_gesture367.png
/kaggle/input/hand-gestures-recognition/dataset/test_set/two/two_gesture662.png
/kaggle/input/hand-gestures-recognition/dataset/test_set/two/two_gesture236.png
/kaggle/input/hand-gestures-recognition/

In [2]:
%cd /kaggle/input/hand-gestures-recognition

/kaggle/input/hand-gestures-recognition


Import Libraries

In [3]:
import os
from pathlib import Path
import cv2
import math
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

Setup the Datasets

In [4]:
#List out the files and folders inside the extracted dataset folder
os.listdir('dataset')

['test_set', 'training_set']

In [6]:
#Declare the path to training and testing datasets
train_path = os.path.join('dataset','training_set')
test_path = os.path.join('dataset','test_set')

In [7]:
print(train_path)
print(test_path)

dataset/training_set
dataset/test_set


In [9]:
# Define the class label
class_label = ['L','four','thumbsup','two','up','three','background'] 

In [10]:
# The function to load all images in each subdirectory
def load_data(subdir, subdir_name, labels, images):
  subdir_path = Path(subdir)

  for file in subdir_path.iterdir():
    if file.is_file():
      labels = np.append(labels,int(class_label.index(subdir_name)))
      file_path = os.path.join(subdir,file.name)
      image = np.expand_dims(cv2.imread(file_path), axis=0)
      if images.size==0:
        images = image
      else:
        images = np.vstack((images, image))

  return labels,images


In [12]:
# The function to load all image files in the specified path
def load_directory(dir):
  dir_path = Path(dir)
  labels = np.array([])
  images = np.array([])

  for subdir in dir_path.iterdir():
    if subdir.is_dir():
      subdir_path = os.path.join(dir,subdir.name)
      labels, images = load_data(subdir_path, subdir.name, labels, images)
  return labels, images

In [ ]:
train_labels, train_images = load_directory(train_path)

In [ ]:
#Check the shape of training dataset's images and length of its labels
print("Shape:",train_images.shape)
print("Length:",len(train_labels))

In [ ]:
#Show the last image in training dataset and check its label
plt.imshow(train_images[10340])
plt.show()
print("Gesture:",class_label[int(train_labels[10340])])

In [ ]:
#Load the testing dataset
test_labels, test_images = load_directory(test_path)

In [ ]:
#Check the shape of testing dataset's images and length of its labels
print("Shape:",test_images.shape)
print("Length:",len(test_labels))

In [ ]:
#Show the first image in testing dataset and check its label
plt.imshow(test_images[0])
plt.show()
print("Gesture:",class_label[int(test_labels[0])])

In [ ]:
#Normalize the images
train_images = train_images/255.0
test_images = test_images/255.0

Setup and Train the Model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,(3,3), activation='relu', input_shape=(90,70,3)),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
    tf.keras.layers.MaxPool2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(7)
])

In [ ]:
#Compile the model
model.compile(optimizer='adam',loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [ ]:
#Fit the model
model.fit(train_images, train_labels, epochs=5)

In [ ]:
#Evaluate the loss and accuracy of the model
test_loss, test_acc = model.evaluate(test_images,test_labels)

In [ ]:
print('\nTest loss:', test_loss)
print('\nTest accuracy:', test_acc)

Test the model

In [ ]:
#Attach the Softmax Layer
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])

In [ ]:
plt.imshow(test_images[0])
plt.show()
print("\nActual digit:",class_label[int(test_labels[0])])

In [ ]:
#The function to plot random X data from the specified model
def predictAndPlotRandomData(model, predict_num):
  rows = math.ceil(math.sqrt(predict_num))
  cols = rows

  rand_index = np.random.randint(len(test_labels)-1, size=predict_num)
  predictions = model.predict(test_images)

  for i in range(predict_num):
    label = np.argmax(predictions[rand_index[i]])
    plt.subplot(rows, cols, i+1)
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(test_images[rand_index[i]])
    true_label = int(test_labels[rand_index[i]])

    if label == true_label:
      color = 'b'
    else:
      color = 'r'

    plt.xlabel("{} {:2.0f}% ({})".format(class_label[label],
                                100*np.max(predictions[rand_index[i]]),
                                class_label[true_label]),
                                color=color, size=7)
  plt.tight_layout()
  plt.show()

In [ ]:
#Call the graph plotting function to plot random X number of data
predictAndPlotRandomData(model,36)